In [5]:
import pandas as pd
inputFileName = "MineScheduling_miniDataSet.xlsx"
paramDF = pd.read_excel(inputFileName, "Param", skiprows=0)
jobDF = pd.read_excel(inputFileName, "Job", skiprows=0)
jobMachineDF = pd.read_excel(inputFileName, "JobMachine", skiprows=0)
switchingDF = pd.read_excel(inputFileName, "Switching", skiprows=0)

### Set the data into dict + compute maxDuration

In [7]:
horizonDuration = paramDF['Duration (day)'][0]

duration = dict()  #duration[vJob][vMachine]
maxDuration = 0.0
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in duration : duration[vJob] = dict() 
    duration[vJob][vMachine] = jobMachineDF['Duration'][iJob]
    if duration[vJob][vMachine] > maxDuration : maxDuration = duration[vJob][vMachine]

### Create your linear program

In [8]:
from pulp import
prob = LpProblem("MineSchedulingProblem", LpMaximize)  

{'Job1': {'Bull': isFirst_Bull__Job1_}, 'Job2': {'Bull': isFirst_Bull__Job2_}, 'Job3': {'Bull': isFirst_Bull__Job3_, 'Dragline': isFirst_Dragline__Job3_}, 'Job4': {'Dragline': isFirst_Dragline__Job4_}, 'Job5': {'Dragline': isFirst_Dragline__Job5_}, 'Job6': {'Dragline': isFirst_Dragline__Job6_}, 'Job7': {'Bull': isFirst_Bull__Job7_, 'Dragline': isFirst_Dragline__Job7_}, 'Job8': {'Dragline': isFirst_Dragline__Job8_}, 'Job9': {'Dragline': isFirst_Dragline__Job9_}, 'Job10': {'Dragline': isFirst_Dragline__Job10_}}


### Add variables

In [9]:
 #allocationVar[vJob][vMachine] & isFirst[vJob][vMachine]
    
allocationVar = dict()
isFirstVar = dict()
for iJob,vJob in enumerate(jobMachineDF['Job']):
    vMachine = jobMachineDF['Machine'][iJob]
    if not vJob in allocationVar : allocationVar[vJob] = dict() 
    if not vJob in isFirstVar : isFirstVar[vJob] = dict() 
    allocationVar[vJob][vMachine] = LpVariable("Allocation[%s][%s]"%(vJob,vMachine), cat='Binary')
    isFirstVar[vJob][vMachine] = LpVariable("isFirst[%s][%s]"%(vMachine, vJob), cat='Binary') 
    
#startVar[vJob] and isOver[vJob]

startVar = dict()
isOverVar = dict()
for vJob in jobDF['Id'] :
    startVar[vJob] = LpVariable("Start[%s]"%(vJob), lowBound=0, upBound=horizonDuration, cat='Continuous')
    isOverVar[vJob] = LpVariable("IsOver[%s]"%(vJob), lowBound=0, cat='Binary')
    
#isSuccessor[vMachine][vJob1-vJob2] = 1 if vJob2 is the direct successor of vJob1

isSuccessorVar = dict()  
for iMachine,vMachine in enumerate(switchingDF['Machine']):
    vJob1 = switchingDF['Job1'][iMachine]
    vJob2 = switchingDF['Job2'][iMachine]
    if not vMachine in isSuccessorVar : isSuccessorVar[vMachine] = dict()
    if not vJob1 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob1] = dict() 
    if not vJob2 in isSuccessorVar[vMachine] : isSuccessorVar[vMachine][vJob2] = dict() 
    isSuccessorVar[vMachine][vJob1][vJob2] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob1, vJob2), cat='Binary')
    isSuccessorVar[vMachine][vJob2][vJob1] = LpVariable("IsSuccessor[%s][%s][%s]"%(vMachine, vJob2, vJob1), cat='Binary')  

### Add objective

In [18]:
Production=jobDF.set_index('Id').to_dict()['Production']
Production
obj = lpSum([Production[i]*isOverVar[i] for i in Production.keys()])

#obj += lpSum(...)

prob += obj
print(prob)

MineSchedulingProblem:
MAXIMIZE
36*IsOver_Job10_ + 58*IsOver_Job1_ + 43*IsOver_Job2_ + 82*IsOver_Job3_ + 40*IsOver_Job4_ + 28*IsOver_Job5_ + 69*IsOver_Job6_ + 34*IsOver_Job7_ + 35*IsOver_Job8_ + 49*IsOver_Job9_ + 0
VARIABLES
0 <= IsOver_Job10_ <= 1 Integer
0 <= IsOver_Job1_ <= 1 Integer
0 <= IsOver_Job2_ <= 1 Integer
0 <= IsOver_Job3_ <= 1 Integer
0 <= IsOver_Job4_ <= 1 Integer
0 <= IsOver_Job5_ <= 1 Integer
0 <= IsOver_Job6_ <= 1 Integer
0 <= IsOver_Job7_ <= 1 Integer
0 <= IsOver_Job8_ <= 1 Integer
0 <= IsOver_Job9_ <= 1 Integer



### Add constraints

In [21]:
#Max one allocation per job
for vJob in jobDF['Id']:
    prob += lpSum([allocationVar[vJob][vMachine] for vMachine in allocationVar[vJob].keys()])<=1


In [41]:
jobMachineDF

,Job,Zone,Trench,Layer,Machine,Duration,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,Job1,Zone1,1,1,Bull,9,NaN,NaN,NaN,NaN,NaN
1,Job2,Zone1,1,2,Bull,14,NaN,NaN,NaN,NaN,NaN
2,Job3,Zone1,1,3,Bull,27,NaN,NaN,NaN,NaN,NaN
3,Job3,Zone1,1,3,Dragline,23,NaN,NaN,NaN,NaN,NaN
4,Job4,Zone1,1,4,Dragline,18,NaN,NaN,NaN,NaN,NaN
5,Job5,Zone1,1,5,Dragline,12,NaN,NaN,NaN,NaN,NaN
6,Job6,Zone1,1,6,Dragline,27,NaN,NaN,NaN,NaN,NaN
7,Job7,Zone1,2,3,Bull,17,NaN,NaN,NaN,NaN,NaN
8,Job7,Zone1,2,3,Dragline,15,NaN,NaN,NaN,NaN,NaN
9,Job8,Zone1,2,4,Dragline,12,NaN,NaN,NaN,NaN,NaN


In [45]:
#Max one "isFirst" job per 

for vMachine in paramDF['Machine']:
        vJobs=jobMachineDF.loc[jobMachineDF['Machine'] == vMachine , 'Job']
        print(vJobs)
        prob += lpSum([isFirstVar[vJob][vMachine] for vJob in vJobs])<=1
    


0    Job1
1    Job2
2    Job3
7    Job7
Name: Job, dtype: object
3      Job3
4      Job4
5      Job5
6      Job6
8      Job7
9      Job8
10     Job9
11    Job10
Name: Job, dtype: object


In [46]:
print(prob)

MineSchedulingProblem:
MAXIMIZE
36*IsOver_Job10_ + 58*IsOver_Job1_ + 43*IsOver_Job2_ + 82*IsOver_Job3_ + 40*IsOver_Job4_ + 28*IsOver_Job5_ + 69*IsOver_Job6_ + 34*IsOver_Job7_ + 35*IsOver_Job8_ + 49*IsOver_Job9_ + 0
SUBJECT TO
_C1: Allocation_Job1__Bull_ <= 1

_C2: Allocation_Job2__Bull_ <= 1

_C3: Allocation_Job3__Bull_ + Allocation_Job3__Dragline_ <= 1

_C4: Allocation_Job4__Dragline_ <= 1

_C5: Allocation_Job5__Dragline_ <= 1

_C6: Allocation_Job6__Dragline_ <= 1

_C7: Allocation_Job7__Bull_ + Allocation_Job7__Dragline_ <= 1

_C8: Allocation_Job8__Dragline_ <= 1

_C9: Allocation_Job9__Dragline_ <= 1

_C10: Allocation_Job10__Dragline_ <= 1

_C11: isFirst_Bull__Job1_ + isFirst_Bull__Job2_ + isFirst_Bull__Job3_
 + isFirst_Bull__Job7_ <= 1

_C12: isFirst_Dragline__Job10_ + isFirst_Dragline__Job3_
 + isFirst_Dragline__Job4_ + isFirst_Dragline__Job5_ + isFirst_Dragline__Job6_
 + isFirst_Dragline__Job7_ + isFirst_Dragline__Job8_ + isFirst_Dragline__Job9_
 <= 1

VARIABLES
0 <= Allocation_Job

In [38]:
#Constraint stating the "start" variable
#Start[j’] >= Start[j] + duration[j][m] + switchingTime[m][j][j’]  - BigM * (1 - isSuccessor[m][j][j’])
#Start[j'] - Start[j] - BigM * isSuccessor[m][j][j'] >= duration[j][m] + switchingTime[m][j][j’] - BigM

for iMachine,vMachine in enumerate(switchingDF['Machine']):
    #XXX
    


SyntaxError: unexpected EOF while parsing (<ipython-input-38-6b7bfc499a0f>, line 7)

In [16]:
#Constraint stating the "isOver" variable
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] <= durationHorizon + BigM * (1 - isOver[j])
#start[j] + Sum[m in isPossible[j] ] duration[j][m] * allocation[j][m] + BigM * isOver[j] <= durationHorizon + BigM

for vJob in jobDF['Id']:
    #XXX


In [17]:
#Constraint stating that to be allocated, a job shall either be the first, or be the successor of another job
# allocationVar[vJob][vMachine] <= isFirst[vJob][vMachine] + sum[otherJobs oJob] isSuccessor[vMachine][oJob][vJob]


for iJob,vJob in enumerate(jobMachineDF['Job']):
    #XXX


In [18]:
#Constraint stating that if isOver = 1, then at least one allocation = 1
for vJob in jobDF['Id']:
    #XXX

In [19]:
#Max one successor per job
for iJob,vJob in enumerate(jobMachineDF['Job']):
    #XXX


### Solve and display results

In [20]:
prob.writeLP("mineSchedulingProblem.lp", writeSOS=1, mip=1)
prob.solve()
print("Status:", LpStatus[prob.status])
print ("Objective = ", value(prob.objective))
varsDict = {}
for v in prob.variables():
    varsDict[v.name] = v.varValue
    if "IsOver" in v.name or "Start" in v.name or "Allocation" in v.name:
        if v.varValue != 0.0 : print(v.name, "=", v.varValue)
        

Status: Optimal
Objective =  2900.0
Allocation_Job11__Bull_ = 1.0
Allocation_Job12__Bull_ = 1.0
Allocation_Job13__SmallDragline_ = 1.0
Allocation_Job14__BigDragline_ = 1.0
Allocation_Job15__BigDragline_ = 1.0
Allocation_Job16__BigDragline_ = 1.0
Allocation_Job17__BigDragline_ = 1.0
Allocation_Job18__SmallDragline_ = 1.0
IsOver_Job11_ = 1.0
IsOver_Job12_ = 1.0
IsOver_Job13_ = 1.0
IsOver_Job14_ = 1.0
IsOver_Job15_ = 1.0
IsOver_Job16_ = 1.0
IsOver_Job17_ = 1.0
Start_Job11_ = 23.904762
Start_Job12_ = 22.0
Start_Job13_ = 6.6153846
Start_Job14_ = 28.933333
Start_Job15_ = 34.0
Start_Job16_ = 27.454545
Start_Job17_ = 12.133333
Start_Job18_ = 50.0
Start_Job19_ = 50.0
